#1

In [ ]:
from pycocotools.coco import COCO as coco
import requests
import os
import sys
import threading


In [ ]:
def makeDirectory(dirName):
  try:
      os.mkdir(dirName)
      print(f"\nMade {dirName} Directory.\n")
  except:
      pass


In [ ]:
def getImagesFromClassName(className):
    makeDirectory(f'downloaded_images/{className}')
    catIds = coco.getCatIds(catNms=[className])
    imgIds = coco.getImgIds(catIds=catIds )
    images = coco.loadImgs(imgIds)

    print(f"Total Images: {len(images)} for class '{className}'")

    for im in images:
        image_file_name = im['file_name']
        label_file_name = im['file_name'].split('.')[0] + '.txt'

        fileExists = os.path.exists(f'downloaded_images/{className}/{image_file_name}')
        if(not fileExists):
            img_data = requests.get(im['coco_url']).content
            annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)
            print(f"{className}. Downloading - {image_file_name}")
            for i in range(len(anns)):
                # Yolo Format: center-x center-y width height
                # All values are relative to the image.
                topLeftX = anns[i]['bbox'][0] / im['width']
                topLeftY = anns[i]['bbox'][1] / im['height']
                width = anns[i]['bbox'][2] / im['width']
                height = anns[i]['bbox'][3] / im['height']

                s = "0 " + str((topLeftX + (topLeftX + width)) / 2) + " " + \
                str((topLeftY + (topLeftY + height)) / 2) + " " + \
                str(width) + " " + \
                str(height)

                if(i < len(anns) - 1):
                    s += '\n'

            with open(f'downloaded_images/{className}/{image_file_name}', 'wb') as image_handler:
                image_handler.write(img_data)
            with open(f'downloaded_images/{className}/{label_file_name}', 'w') as label_handler:
                label_handler.write(s)
        else:
           print(f"{className}. {image_file_name} - Already Downloaded.")

#2

In [ ]:
!git clone https://github.com/KaranJagtiani/YOLO-Coco-Dataset-Custom-Classes-Extractor

Cloning into 'YOLO-Coco-Dataset-Custom-Classes-Extractor'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 4), reused 9 (delta 1), pack-reused 0
Receiving objects: 100% (22/22), 6.11 KiB | 6.11 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2023-10-08 14:51:26--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.242, 3.5.20.112, 52.217.135.49, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.242|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  92.7MB/s    in 2.6s    

2023-10-08 14:51:28 (92.7 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [ ]:
from pycocotools.coco import COCO
import os
import shutil
import json
annotation_file = 'path/to/annotations/instances_train2017.json'
image_dir = 'path/to/images/train2017/'
coco = COCO(annotation_file)
included_categories = ['person', 'dog', 'car']
included_category_ids = coco.getCatIds(catNms=included_categories)
image_ids = []
for cat_id in included_category_ids:
    image_ids.extend(coco.getImgIds(catIds=[cat_id]))
image_ids = list(set(image_ids))
subset_annotations = coco.loadImgs(image_ids)
output_image_dir = 'path/to/output/images/'
os.makedirs(output_image_dir, exist_ok=True)
for img_info in subset_annotations:
    img_file_name = img_info['file_name']
    img_path = os.path.join(image_dir, img_file_name)
    shutil.copy(img_path, output_image_dir)
subset_annotations_json = coco.loadAnns(coco.getAnnIds(imgIds=image_ids))
output_annotations_file = 'path/to/output/annotations.json'
with open(output_annotations_file, 'w') as outfile:
    json.dump(subset_annotations_json, outfile)